In [1]:
import itertools

import cirq
import numpy as np
import sys
sys.path.append("..")

from sim.circuit import gen_1d_chain_xeb_random_circuit
from sim.simulate import large_scale_xeb_sim

In [2]:
exponents = np.linspace(0, 7/4, 8)
SINGLE_QUBIT_GATES = tuple(
    cirq.PhasedXZGate(x_exponent=0.5, z_exponent=z, axis_phase_exponent=a)
    for a, z in itertools.product(exponents, repeat=2)
)

MAX_DEPTH = 400
cycle_depths = list(range(1, 21)) + [25, 35, 40]
e_pauli = 1e-2
e_dep = e_pauli / (1 - 1/2**2)
noise_model = cirq.devices.noise_model.ConstantQubitNoiseModel(cirq.depolarize(e_pauli))
save_resume_filepath = "/home/cuquantum/xebsim/result/1d_chain/result.csv"

In [3]:
res = {}
ns = [6, 8, 10]
# ns = [12]
n_circuits = 10
for n in ns:
    qubits = cirq.LineQubit.range(n)
    circuits = [
        gen_1d_chain_xeb_random_circuit(
            qubits,
            depth=MAX_DEPTH,
            single_qubit_gates=SINGLE_QUBIT_GATES,
            two_qubit_op_factory=lambda a, b, _: cirq.SQRT_ISWAP(a, b)
        )
        for _ in range(n_circuits)
    ]

    cycle_xebs = [
        large_scale_xeb_sim(circuit, noise_model, cycle_depths=cycle_depths)
        for circuit in circuits
    ]
    res[n] = cycle_xebs

8 qubits noisy state simulation:  86%|████████▋ | 70/81 [00:02<00:00, 31.06it/s]


KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt

for n in ns:
    cycle_xebs = np.mean(res[n], axis=0)
    plt.plot(cycle_depths, cycle_xebs, label=f'{n} qubits', marker='o')
plt.yscale('log')
plt.yticks(np.logspace(-2, 4, num=7))
plt.legend()
plt.title('1D chain')
plt.xlabel("Depth")
plt.ylabel("Linear XEB")
plt.show()


NameError: name 'ns' is not defined

In [5]:
from sim.utils import _fit_exponential_decay

_fit_exponential_decay(cycle_depths[15:], np.mean(res[6], axis=0)[15:])

/media/disk/Supremacy/xebsim/.venv/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:49: RuntimeWarning: invalid value encountered in scalar divide
  t2 = c / q


(0.9999999999999999,
 0.9062271474006709,
 0.3768167510353321,
 0.01864689434307791)

In [10]:
(1 - e_dep) ** 6

0.922619730810557